# The Dual Approach to Sparse Regression
## A Comparison with LASSO

### Introduction
We have seen in class different formulations of robust and sparse regression. In particular, we saw how LASSO regression can be interpreted as a particular instance of robust regression, and we have seen both a primal and a dual approach to sparse regression. In this recitation, we will implement the dual approach, and compare it with LASSO regression.




### The Dual Method

We showed in the slides that our original problem,

\begin{align*}
&\min_\boldsymbol\beta \frac{1}{2}\| \mathbf{y} - \mathbf{X}\boldsymbol\beta \|_2^2+\frac{1}{2\gamma}\|\boldsymbol\beta \|_2^2.\\
&\text{s.t.}\; \|\boldsymbol\beta\|_0\leq k
\end{align*}
 can be transformed to:
\begin{align*}
&\min_{\boldsymbol s \in S_k} \frac{1}{2}\mathbf{y}^T\left(\mathbf{I}_n+\gamma\sum_i s_i\mathbf{K}_i\right)^{-1}\mathbf{y}
\end{align*}
where $K_i=X_iX_i^T$, and $X_i$ is the $i$th column of the feature matrix. Now we have a convex binary problem that is convex, as it is a composition of convex functions. Therefore, we can utilize the cutting plane method to solve it. 

In [1]:
using Gurobi, StatsBase, DataFrames, JuMP, LinearAlgebra, Distributions,Random

### The Cutting Plane

To use the cutting plane method to solve this problem, we need to know how to quickly calculate the value and the derivative of the function:
\begin{align*}
c(\mathbf{s})=&\frac{1}{2}\mathbf{y}^T\left(\mathbf{I}_n+\gamma\sum_i s_i\mathbf{K}_i\right)^{-1}\mathbf{y}
\end{align*}
With respect to $\mathbf{s}$ in the minimization problem above. Some clever algebra leads to the solution that:
\begin{align*}
&\nabla c(\mathbf{s})_j = -\frac{\gamma}{2}\alpha^{*T}X_jX_j^T\alpha^*
\end{align*}
Where $\alpha^*=\left(\mathbf{I}_n-\mathbf{X}_s\left(\frac{\mathbf{I}_k}{\gamma}+\mathbf{X}_s^T\mathbf{X}_s\right)^{-1}\mathbf{X}_s^T\right)\mathbf{y}$. The Matrix Inversion Lemma further tells us that:
$$
\left(\mathbf{I}_n+\gamma\sum_i s_i\mathbf{K}_i\right)^{-1}=\left(\mathbf{I}_n-\mathbf{X}_s\left(\frac{\mathbf{I}_k}{\gamma}+\mathbf{X}_s^T\mathbf{X}_s\right)^{-1}\mathbf{X}_s^T\right)
$$
So $c(\mathbf{s})=\frac{1}{2}y^T\alpha^*$.

Therefore, we have:
\begin{align*}
c(\mathbf{s})&=\frac{1}{2}y^T\alpha^*\\
\nabla c(\mathbf{s})_j &= -\frac{\gamma}{2}\alpha^{*T}X_jX_j^T\alpha^*
\end{align*}
We implement this below.

In [2]:
function regobj(X,Y,s,γ)
    indices = findall(s .> 0.5)
    n = length(Y)
    denom = 2n
    Xs = X[:, indices]
    α = Y - Xs * (inv(I / γ + Xs' * Xs) * (Xs'* Y))
    val = dot(Y, α) / denom
    tmp = X' * α
    ∇s = -γ .* tmp .^ 2 ./ denom
  return val, ∇s
end

regobj (generic function with 1 method)

### Setting Up the Model

With this helper function, we will now start building the dual method for sparse regression. 

#### Step 1: Define the Variables:

In [3]:
p = 100
Random.seed!(15095)
X = rand(1000,100)
Random.seed!(15095)
Y = rand(1000)
k = 5
γ = 100
miop = Model(Gurobi.Optimizer)
# Optimization variables
@variable(miop, s[1:p], Bin)
@variable(miop, t >= 0)

Academic license - for non-commercial use only


t

#### Step 2: Set Up Constraints and Objective

In [4]:
# Constraints
@constraint(miop, sum(s) <= k)
s0 = zeros(p)
s0[1:k] .= 1
p0, ∇s0 = regobj(X,Y, s0, γ)
@constraint(miop, t >= p0 + dot(∇s0, s - s0))
# objective

@objective(miop, Min, t)

t

#### Step 3: Define the Cutting Plane Function

In [5]:
function outer_approximation(cb_data)
    s_val = []
    for i = 1:p
        s_val = [s_val;callback_value(cb_data, s[i])]
    end
    #s_val = callback_value.(cb_data, s)
    obj, ∇s = regobj(X,Y, s_val, γ)
    # add the cut: t >= obj + sum(∇s * (s - s_val))
    offset = sum(∇s .* s_val)
    con = @build_constraint( t >= obj + sum(∇s[j] * s[j] for j=1:p) - offset)
    MOI.submit(miop, MOI.LazyConstraint(cb_data), con)
    #@lazyconstraint(cb, t >= obj + sum(∇s[j] * s[j] for j=1:p) - offset)
end
MOI.set(miop, MOI.LazyConstraintCallback(), outer_approximation)

#s_val = JuMP.value.(s)


outer_approximation (generic function with 1 method)

#### Step 4: Modularize the Code into a Function

In [6]:
function SparseRegression(X,Y,γ,k; solver_output=0)
    p=size(X)[2]
    miop = Model(Gurobi.Optimizer)
    set_optimizer_attribute(miop, "OutputFlag", solver_output) 
    # Optimization variables
    @variable(miop, s[1:p], Bin)
    @variable(miop, t >= 0)
    # Objective
    @objective(miop, Min, t)
    # Constraints
    @constraint(miop, sum(s) <= k)
    s0 = zeros(p)
    s0[1:k] .= 1
    p0, ∇s0 = regobj(X, Y, s0, γ)
    @constraint(miop, t >= p0 + dot(∇s0, s - s0))
    function outer_approximation(cb_data)
        s_val = []
        for i = 1:p
            s_val = [s_val;callback_value(cb_data, s[i])]
        end
        #s_val = callback_value.(cb_data, s)
        obj, ∇s = regobj(X,Y, s_val, γ)
        # add the cut: t >= obj + sum(∇s * (s - s_val))
        offset = sum(∇s .* s_val)
        con = @build_constraint( t >= obj + sum(∇s[j] * s[j] for j=1:p) - offset)
        MOI.submit(miop, MOI.LazyConstraint(cb_data), con)
        #@lazyconstraint(cb, t >= obj + sum(∇s[j] * s[j] for j=1:p) - offset)
    end
    MOI.set(miop, MOI.LazyConstraintCallback(), outer_approximation)
    # solving the actual model
    optimize!(miop)
    s_opt = JuMP.value.(s)
    s_nonzeros = findall(x -> x>0.5, s_opt)
    β = zeros(p)
    X_s = X[:, s_nonzeros]
    # formula for the nonzero coefficients
    β[s_nonzeros] = γ * X_s' * (Y - X_s * ((I / γ + X_s' * X_s) \ (X_s'* Y)))
    return s_opt, β,s_nonzeros
end

SparseRegression (generic function with 1 method)

#### Step 5: Load Data to Verify

In [7]:
using CSV
train_data = CSV.read("Train lpga2008_opt.csv",DataFrame; header=false)


,Column1,Column2,Column3,Column4,Column5,Column6,Column7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.4673,225.1,78.2,55.0,29.47,1.26,47.9
2,9.4293,242.4,65.5,66.1,28.51,0.76,23.4
3,9.4986,242.4,70.7,67.7,28.64,0.7,48.4
4,9.476,257.1,67.3,68.2,29.08,0.77,37.3
5,7.6488,238.1,71.7,57.2,27.58,0.95,38.6
6,7.3895,261.1,60.9,62.3,29.34,1.02,33.9
7,9.1652,261.1,68.7,65.3,28.14,1.14,40.2
8,7.4426,252.0,70.7,62.3,29.67,1.16,39.7
9,7.3775,238.2,65.3,61.4,30.04,1.05,36.2


In [8]:
y_train = Vector(train_data[:,1])
X_train = Array{Float64,2}(train_data[:,2:end])
s_opt, β,s_nonzeros  = SparseRegression(X_train,y_train,1,3)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


([1.0, -0.0, 1.0, 1.0, -0.0, -0.0], [0.0232123, 0.0, 0.200118, -0.356159, 0.0, 0.0], [1, 3, 4])

### Comparison with LASSO

Now that we have set this up, let us compare the speed and the quality of the solution with LASSO. To do so, we will use the Lasso regression function you coded in the previous recitation, tested agains the Sparse Regression method coded above.

We will use simulated data to do the comparison.

In [9]:
# Defining parameters
n = 10
p = 10
k = 1
β = zeros(p)

# Picking sparse dimensions
Random.seed!(15095)
nonzeroinds = sample(1:p,k,replace=false)
Random.seed!(15095)
β[nonzeroinds] = rand(k)

# Generating the Data
Random.seed!(15095)
X = rand(n,p)
Random.seed!(15095)
y = X * β + rand(Normal(0,0.1),n)

10-element Array{Float64,1}:
  0.05023161516038392
  0.17663664167377974
  0.05261684292865973
  0.21301142847894008
  0.16932567199372933
  0.22068937224277466
  0.1554644960102942 
 -0.11331278812919703
  0.1446216883617416 
  0.15966403077481892

In [10]:
s_opt, β_opt = SparseRegression(X,y,100,k)
#s_nonzeros = findall(x -> x>0.5, s_opt)
#println(sort(s_nonzeros))
#println(sort(nonzeroinds))
#println(mean((abs.(β-β_opt)./β)[nonzeroinds]))

Academic license - for non-commercial use only
Academic license - for non-commercial use only


([-0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.213629], [10])

### Performance of Sparse Regression

#### Accuracy

In [11]:
s_nonzeros = findall(x -> x>0.5, s_opt)
println(sort(s_nonzeros))
println(sort(nonzeroinds))
println(mean((abs.(β-β_opt)./β)[nonzeroinds]))
println("Sparse regression finds the coefficients ",β_opt)
println("The real coefficients are ",β)

[10]
[10]
0.09161686550545232
Sparse regression finds the coefficients [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.213629]
The real coefficients are [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.235175]


#### Speed

In [12]:
@time SparseRegression(X,y,100,k)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
  0.015363 seconds (8.52 k allocations: 386.453 KiB)


([-0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.213629], [10])

Here, we define the Lasso regression function using the previous recitation.

In [13]:
function l1_regression(X, y, rho; solver_output=0)
    n,p = size(X)
    
    # Build model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Insert variables and constraints
    @variable(model,beta[j=1:p])
    @variable(model,beta_abs[j=1:p]>=0)
    @variable(model, sse>=0)
    @constraint(model,[j=1:p], beta[j]<=beta_abs[j])
    @constraint(model,[j=1:p], beta[j]>=-beta_abs[j])
    @constraint(model, sum((y[i]-sum(X[i,j]*beta[j] for j=1:p))^2 for i=1:n) <= sse)
    @objective(model,Min, sse + rho*sum(beta_abs[j] for j=1:p))
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
end

l1_regression (generic function with 1 method)

#### Lasso Accuracy

In [14]:
beta_lasso = l1_regression( X, y, 1)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


10-element Array{Float64,1}:
 4.1210382477950615e-10
 6.455406863317954e-10 
 2.921521154348343e-9  
 2.5230180511657014e-9 
 2.0947335613004685e-9 
 0.01778555064230097   
 1.7515418661040304e-9 
 1.90371305175229e-9   
 1.3788150059355628e-9 
 0.08392324035684719   

In [15]:
s_nonzeros = findall(x -> abs.(x)>0, beta_lasso)
println(sort(s_nonzeros))
println(mean((abs.(β-beta_lasso)./β)[nonzeroinds]))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
0.643144906484792


We compare this back to the sparse regression formulation:

In [16]:
println(mean((abs.(β-β_opt)./β)[nonzeroinds]))

0.09161686550545232


In this case, lasso fails to recover the true coefficients and results in a large error, while sparse regression pretty much extracts the exact true solution. 

#### Lasso Speed

In [17]:
@time l1_regression( X, y, 3.0)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
  0.160892 seconds (265.30 k allocations: 13.393 MiB, 7.29% gc time)


10-element Array{Float64,1}:
 3.213437752145101e-9 
 7.167077745923063e-9 
 2.5394354229264988e-8
 4.2966141093543934e-8
 3.9223465661398344e-8
 1.0959548320103353e-7
 2.5567934132299935e-8
 1.979773980874511e-8 
 1.751294166754463e-8 
 1.0677548893917954e-7

We compare this back to the sparse regression formulation again:

In [18]:
@time SparseRegression(X,y,100,k)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
  0.009731 seconds (8.51 k allocations: 386.406 KiB)


([-0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.213629], [10])

The times for these functions can actually vary across different runs, but generally we see that the Lasso is either slower than or runs at a comparable speed to sparse regression. Sparse regression, however, gets much closer to recovering the true coefficient compared to Lasso. Try out different combination of $n$, $p$, and $k$ for yourself to see what works best for each case.